In [ ]:
import torch
from kornia.color import lab_to_rgb, rgb_to_lab, hsv_to_rgb, rgb_to_hsv
from matplotlib import pyplot as plt
class ImageContraster():
    def __init__(self):
        pass

    def Hist(self,img_2, bins):
        hist = torch.histc(img_2, bins=bins, min=0, max=bins - 1)
        return hist

    def Hist_after_mapping(self,hist):
        number = torch.sum(hist)
        hist_am = torch.cumsum(hist, dim=0) / number * 100
        # hist_am = hist_am.clone().type(torch.int)
        return hist_am

    def mapping(self,img_2, bins,space="LAB"):
        hist = self.Hist(img_2, bins)
        hist_am = self.Hist_after_mapping(hist)
#         print(hist, hist_am)
        #mapping怎么写？
        if space == "LAB":            
            for i in range(0, bins):
                img_2 = torch.where(torch.tensor(img_2/100*(bins-1)).type(torch.int) == torch.tensor(i), hist_am[i], img_2)
        if space == "HSV":            
            for i in range(0, bins):
                img_2 = torch.where(img_2.type(torch.int) == i * (bins-1), hist_am[i], img_2) / (bins-1)
        
        return img_2

    def HE(self,img,bins,space="LAB"):
        # global img_2
        img = torch.unsqueeze(img,0)
        if space == "LAB":
            img = rgb_to_lab(img)
            img[:,0,:,:] = img[:,0,:,:]
#             print(img[:,0,:,:])
            img[:,0,:,:] = self.mapping(img[:,0,:,:], bins)
            img = lab_to_rgb(img)
        elif space == "HSV":
            img = rgb_to_hsv(img)
            img[:,2,:,:] = img[:,2,:,:] * (bins-1)
            img[:,2,:,:] = self.mapping(img[:,2,:,:], bins) / 100 
            img = hsv_to_rgb(img)
        return img[0]
    
    def show_from_tensor(self, tensor, title=None):
        img = tensor.clone().permute(1,2,0)
        img = img.cpu().numpy()
        plt.figure()
        plt.imshow(img)
        if title is not None:
            plt.title(title)
        plt.pause(0.001)


In [ ]:
#-*- coding : utf-8 -*-

from PIL import Image, ImageOps
import numpy as np
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# from contrast import ImageContraster
# from HE_Methods import ImageContraster
# read image
loader = transforms.Compose([transforms.ToTensor()])  
img = Image.open("ep1_2023-01-17 02_42_00.206.png")
img = loader(img).to(device)
# print(img)
# contraster
icter = ImageContraster()
# HE
he_eq_img = icter.HE(img, 256, space='LAB')

icter.show_from_tensor(he_eq_img, title=None)
icter.show_from_tensor(img, title=None)
print(torch.max(torch.abs(img-he_eq_img)))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

img_origin = cv2.imread('ep1_2023-01-17 02_15_07.399.png')
img = cv2.imread('ep1_2023-01-17 02_42_00.206.png') 
img_HE = cv2.imread('HE.png') 
img_AHE = cv2.imread('AHE.png') 
img_CLAHE = cv2.imread('CLAHE.png') 
img_LRSHE = cv2.imread('LRSHE.png') 

img_origin_HSV = cv2.cvtColor(img_origin, cv2.COLOR_BGR2HSV)
img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_HE_HSV = cv2.cvtColor(img_HE, cv2.COLOR_BGR2HSV)
img_AHE_HSV = cv2.cvtColor(img_AHE, cv2.COLOR_BGR2HSV)
img_CLAHE_HSV = cv2.cvtColor(img_CLAHE, cv2.COLOR_BGR2HSV)
img_LRSHE_HSV = cv2.cvtColor(img_LRSHE, cv2.COLOR_BGR2HSV)

histSize = 256
histRange = (0, 256) 
hist_origin = cv2.calcHist([img_origin_HSV], [2], None, [histSize], histRange) 
hist = cv2.calcHist([img_HSV], [2], None, [histSize], histRange) 
hist_HE = cv2.calcHist([img_HE_HSV], [2], None, [histSize], histRange)
hist_AHE = cv2.calcHist([img_AHE_HSV], [2], None, [histSize], histRange)
hist_CLAHE = cv2.calcHist([img_CLAHE_HSV], [2], None, [histSize], histRange)
hist_LRSHE = cv2.calcHist([img_LRSHE_HSV], [2], None, [histSize], histRange)
 
#显示图像
fig,ax = plt.subplots(6,2,figsize=(30,40))
ax[0,0].set_title('origin',fontsize=25)
ax[0,0].imshow(img_origin)
ax[0,1].set_title('hist',fontsize=25)
ax[0,1].plot(hist_origin)

ax[1,0].set_title('after SAnorm',fontsize=25)
ax[1,0].imshow(img)
ax[1,1].set_title('hist',fontsize=25)
ax[1,1].plot(hist)

ax[2,0].set_title('HE',fontsize=25)
ax[2,0].imshow(img_HE)
ax[2,1].set_title('HE hist',fontsize=25)
ax[2,1].plot(hist_HE)

ax[3,0].set_title('AHE',fontsize=25)
ax[3,0].imshow(img_AHE)
ax[3,1].set_title('hist_AHE',fontsize=25)
ax[3,1].plot(hist_AHE)

ax[4,0].set_title('CLAHE',fontsize=25)
ax[4,0].imshow(img_CLAHE)
ax[4,1].set_title('hist_CLAHE',fontsize=25)
ax[4,1].plot(hist_CLAHE)

ax[5,0].set_title('LRSHE',fontsize=25)
ax[5,0].imshow(img_LRSHE)
ax[5,1].set_title('hist_LRSHE',fontsize=25)
ax[5,1].plot(hist_LRSHE)

ax[0,0].axis('off');ax[1,0].axis('off')#关闭坐标轴显示
ax[2,0].axis('off');ax[3,0].axis('off')
ax[4,0].axis('off');ax[5,0].axis('off')
plt.show()  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

plt.rc('font',family='Youyuan',size='9')
plt.rc('axes',unicode_minus='False')
 
img_src = cv2.imread('ep1_2023-01-17 02_42_00.206.png',0) 
img_dst = cv2.equalizeHist(img_src) 
 
histSize = 512
histRange = (0, 256)  
hist_src = cv2.calcHist([img_src], [0], None, [histSize], histRange) 
hist_dst = cv2.calcHist([img_dst], [0], None, [histSize], histRange)  
 
#显示图像
fig,ax = plt.subplots(2,2)
ax[0,0].set_title('hist_src')
ax[0,0].plot(hist_src) 
ax[0,1].set_title('hist_dst')
ax[0,1].plot(hist_dst)
ax[1,0].set_title('src')
ax[1,0].imshow(cv2.cvtColor( img_src, cv2.COLOR_BGR2RGB),'gray')
ax[1,1].set_title('dst') 
ax[1,1].imshow(cv2.cvtColor(img_dst,cv2.COLOR_BGR2RGB),'gray')  
#ax[0,0].axis('off');ax[0,1].axis('off');
ax[1,0].axis('off');ax[1,1].axis('off')#关闭坐标轴显示
plt.show()  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

img_origin = cv2.imread('ep1_2023-01-17 02_15_09.444.png')
img = cv2.imread('ep1_2023-01-17 02_42_02.657.png')  

histSize = 256
histRange = (0, 256) 
hist_origin_B = cv2.calcHist([img_origin], [0], None, [histSize], histRange) 
hist_origin_G = cv2.calcHist([img_origin], [1], None, [histSize], histRange) 
hist_origin_R = cv2.calcHist([img_origin], [2], None, [histSize], histRange)
hist_B = cv2.calcHist([img], [0], None, [histSize], histRange) 
hist_G = cv2.calcHist([img], [1], None, [histSize], histRange) 
hist_R = cv2.calcHist([img], [2], None, [histSize], histRange)
 
#显示图像
fig,ax = plt.subplots(4,2,figsize=(12,16))
ax[0,0].set_title('origin',fontsize=15)
ax[0,0].imshow(img_origin)
ax[0,1].set_title('B',fontsize=15)
ax[0,1].plot(hist_origin_B)
ax[1,0].set_title('G',fontsize=15)
ax[1,0].plot(hist_origin_G, color='green')
ax[1,1].set_title('R',fontsize=15)
ax[1,1].plot(hist_origin_R, color='red')

ax[2,0].set_title('after SAnorm',fontsize=15)
ax[2,0].imshow(img)
ax[2,1].set_title('B',fontsize=15)
ax[2,1].plot(hist_B)
ax[3,0].set_title('G',fontsize=15)
ax[3,0].plot(hist_G, color='green')
ax[3,1].set_title('R',fontsize=15)
ax[3,1].plot(hist_R, color='red')


ax[0,0].axis('off')#关闭坐标轴显示
ax[2,0].axis('off')
plt.show()  